In [ ]:
!pip install 'tensorflow==1.15.0'
import tensorflow as tf
tf.__version__

In [ ]:

import random
import json
import gym
import pandas as pd
import numpy as np



import tensorflow as tf
from kaggle_environments import make, evaluate
from gym import spaces

!apt-get update
!apt-get install -y cmake libopenmpi-dev python3-dev zlib1g-dev
!pip install "stable-baselines[mpi]==2.9.0"

from stable_baselines import PPO2
from stable_baselines.common.policies import MlpPolicy




#===============================================================================




In [ ]:
# Cube data structure

def L(cube):
    # L : 1 -> 3, 3 -> 7, 7 -> 5, 5 -> 1
    temp = cube[1]
    cube[1] = cube[5]
    cube[5] = cube[7]
    cube[7] = cube[3]
    cube[3] = temp
  
def R(cube):
    # R : 2 -> 4, 4 -> 8, 8 -> 6, 6 -> 2
    temp = cube[2]
    cube[2] = cube[6]
    cube[6] = cube[8]
    cube[8] = cube[4]
    cube[4] = temp

def D(cube):
    # D : 3 -> 4, 4 -> 8, 8 -> 7, 7 -> 3
    temp = cube[3]
    cube[3] = cube[7]
    cube[7] = cube[8]
    cube[8] = cube[4]
    cube[4] = temp

def U(cube):
    # U: 1 -> 2, 2 -> 6, 6 -> 5, 5 -> 1
    temp = cube[1]
    cube[1] = cube[5]
    cube[5] = cube[6]
    cube[6] = cube[2]
    cube[2] = temp
    
def F(cube):
    # F : 1 -> 2, 2 -> 4, 4 -> 3, 3 -> 1
    temp = cube[1]
    cube[1] = cube[3]
    cube[3] = cube[4]
    cube[4] = cube[2]
    cube[2] = temp

def B(cube):
    # B : 5 -> 6, 6 -> 8, 8 -> 7, 7 -> 5
    temp = cube[5]
    cube[5] = cube[7]
    cube[7] = cube[8]
    cube[8] = cube[6]
    cube[6] = temp


    
# 6 cubes actions

def is_solved(input_cube):
    solved_cube =[0, 1, 2, 3, 4, 5, 6, 7, 8]
    if (input_cube == solved_cube):
        return True
    else:
        return False



In [ ]:
class CubeEnv(gym.Env):
    """A rubix cube environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self):
        super(CubeEnv, self).__init__()
        #self.solved_cube = ['XXX','BYR', 'YGR', 'RGW', 'BRW', 'BOY', 'OGY', 'BOW', 'OWG']
        self.solved_cube = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        
        self.action_space = spaces.Discrete(6) # 6 possible moves
        #self.observation_space = spaces.Discrete(len(self.solved_cube)) # 8 pieces on the cube      
        self.observation_space = spaces.Box(low=0, high=8, shape=(1,9), dtype=np.int8)
      
        self.solved = 0;
        #  
        
       # self.reset()

    def reset(self):
        # Reset the state of the environment to an initial state
        self.cube = [0, 1, 2, 3, 4, 5, 6, 7, 8]
        self.steps = 0
                
        # Reset - Start from solved cube and do 10-20 random actions
        scramble_steps = random.randint(1,5)
        
        for i in range(scramble_steps):
            direction = random.randint(1,6)
           # print(self.cube, direction)
            if (direction == 1):
                R(self.cube)
            if (direction == 2):
                L(self.cube)
            if (direction == 3):
                B(self.cube)
            if (direction == 4):
                D(self.cube)
            if (direction == 5):
                F(self.cube)
            if (direction == 6):
                U(self.cube)
        
        self.start_cube = self.cube[:]        
        self.obs = self.cube
        return np.array([self.obs])
    

    def step(self, action):
        # Execute one time step within the environment
       
        # Do "action" in the environment
        if (action == 1):
             R(self.cube)
        if (action == 2):
             L(self.cube)
        if (action == 3):
             B(self.cube)
        if (action == 4):
             D(self.cube)
        if (action == 5):
             F(self.cube)
        if (action == 6):
             U(self.cube)

        # Increase steps
        self.steps += 1

        reward = 0
        # Calculate reward
        if (is_solved(self.cube)):
            reward = 1000
            print("Solved!! Steps=", self.steps, "Start=", self.start_cube)
            done = True
            self.solved += 1
        else:
            if (self.steps > 200):
                #Quits += 1
                done = True
            else:
                reward = -1
                done = False

        obs = np.array([self.cube])

        return obs, reward, done, {}


#    def _on_training_end(self):
#        """
#        This event is triggered before exiting the `learn()` method.
#        """
#        print("Solved = ", self.solved)
    

    def render(self, mode='human', close=False):
        # Render the environment to the screen
        
        print(f'Step: {self.steps}')
        print(f'Cube: {self.cube}')
        


In [ ]:
from stable_baselines.common.vec_env import DummyVecEnv
cube1 = CubeEnv()
cube_env = DummyVecEnv([lambda: cube1])


In [ ]:
from stable_baselines.ppo2 import PPO2
from stable_baselines.common.schedules import LinearSchedule
from stable_baselines.common.policies import MlpPolicy
# defining your network

policy_kwargs = dict(act_fun=tf.nn.relu, net_arch=[128, 128])
cube_model = PPO2('MlpPolicy',  cube_env, verbose=0 #, 
                  # policy_kwargs=policy_kwargs, 
                  # learning_rate=LinearSchedule( 40000*5, initial_p=0.0005, final_p=0.00005).value, 
                  #tensorboard_log="./mlp"
                 )

In [ ]:
# auxiliary function to evaluate model
def evaluate(model, env, num_steps=1000):
  """
  Evaluate a RL agent
  :param model: (BaseRLModel object) the RL Agent
  :param num_steps: (int) number of timesteps to evaluate it
  :return: (float) Mean reward for the last 100 episodes
  """
  episode_rewards = [0.0]
  obs = env.reset()
 #! print("obs = ", obs)
  
  solved = 0;
  for i in range(num_steps):
      # _states are only useful when using LSTM policies
      action, _states = model.predict(obs)
      # print("action = ", action)
      # here, action, rewards and dones are arrays
      # because we are using vectorized env
      obs, rewards, dones, info = env.step(action)
      #! print("obs = ", obs, "action =", action, "rewards = ", rewards, "dones = ", dones)
      
      if rewards[0] == 1000:
            solved += 1
            
      # Stats
      episode_rewards[-1] += rewards[0]
      if dones[0]:
          #print("Solved!! obs = ", obs)
          
          obs = env.reset()
          episode_rewards.append(0.0)
  # Compute mean reward for the last 100 episodes
  mean_100ep_reward = round(np.mean(episode_rewards[-100:]), 1)
  print("Mean reward:", mean_100ep_reward, "Num of episodes:", len(episode_rewards), "Num times solved:", solved)
  
  return mean_100ep_reward

In [ ]:
#evaluate(cube_model, cube_env, num_steps = 10)

In [ ]:
#import time
#start = time.time()
#end = time.time()
#print('training: ', end-start)

evaluate(cube_model, cube_env, num_steps = 10000)

cube_model.learn(500000)
#print("Quits = ", Quits, "Solves = ",Solves)



In [ ]:
evaluate(cube_model, cube_env, num_steps = 10000)


In [ ]:
Bigger NN ?
Better definintion of observation space

In [ ]:
env = YourEnv()
obs = env.reset()
n_steps = 10
for _ in range(n_steps):
    # Random action
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)